In [2]:
import numpy as np
from tensorflow import keras

In [3]:
def random_generator(length):
    return [np.random.rand() for _ in range(length)]

In [28]:
seq = random_generator(5)

In [35]:
def cumulative_sum(seq):
    seq = np.pad(seq,(1,0),mode='constant')
    for i in range(1,len(seq)):
        seq[i] = seq[i]+seq[i-1]
    return seq[1:]

In [43]:
def out(cum_seq):
    test = len(cum_seq)/4.0
    y = list()
    for i in cum_seq:
        if i < test:
            y.append(0)
        else:
            y.append(1)
    return y


In [88]:
def generate_example(n_patterns,length):
    X,y = list(),list()
    for _ in range(n_patterns):
        seq = random_generator(length)
        cum_seq = cumulative_sum(seq)
        out_seq = out(cum_seq)
        X.append(seq)
        y.append(out_seq)
    X = np.array(X).reshape(n_patterns,length,1)
    y = np.array(y).reshape(n_patterns,length,1)
    return X,y

In [89]:
#define model
time_step = 10
model = keras.models.Sequential()
model.add(keras.layers.Bidirectional(keras.layers.LSTM(50,return_sequences=True),input_shape=(time_step,1)))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(1,activation='sigmoid')))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_5 (Bidirection (None, 10, 100)           20800     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 10, 1)             101       
Total params: 20,901
Trainable params: 20,901
Non-trainable params: 0
_________________________________________________________________
None


In [90]:
X,y = generate_example(50000,time_step)
model.fit(X,y,batch_size=10,epochs=1)

Epoch 1/1
50000/50000 [==============================] - 43s 863us/step - loss: 0.0521 - acc: 0.9804


In [92]:
# make predictions
for _ in range(10):
    X, y = generate_example(1, time_step)
    yhat = model.predict_classes(X, verbose=0)
    exp, pred = y.reshape(time_step), yhat.reshape(time_step)
    print( 'y=%s, yhat=%s, correct=%s' % (exp, pred, np.array_equal(exp,pred)))

y=[0 0 0 1 1 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=False
y=[0 0 0 0 0 1 1 1 1 1], yhat=[0 0 0 0 0 1 1 1 1 1], correct=True
y=[0 0 0 0 1 1 1 1 1 1], yhat=[0 0 0 0 0 1 1 1 1 1], correct=False
y=[0 0 0 0 0 1 1 1 1 1], yhat=[0 0 0 0 0 1 1 1 1 1], correct=True
y=[0 0 0 0 0 0 1 1 1 1], yhat=[0 0 0 0 0 0 1 1 1 1], correct=True
y=[0 0 1 1 1 1 1 1 1 1], yhat=[0 0 1 1 1 1 1 1 1 1], correct=True
y=[0 0 0 1 1 1 1 1 1 1], yhat=[0 0 0 1 1 1 1 1 1 1], correct=True
y=[0 0 0 0 1 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=True
y=[0 0 0 0 0 0 1 1 1 1], yhat=[0 0 0 0 0 0 1 1 1 1], correct=True
y=[0 0 0 1 1 1 1 1 1 1], yhat=[0 0 0 1 1 1 1 1 1 1], correct=True
